<a href="https://colab.research.google.com/github/HungNguyen501/Multi-Text-Classification-NLP-/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# py train.py lstm 50 data/train/data.xlsx
import tensorflow as tf 
import keras
import nltk
import pandas as pd
import numpy as np
import re
import codecs
import pickle
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import sys
from datetime import datetime
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer
from google.colab import drive

In [ ]:
model_name = 'bilstm'
EMBEDDING_DIM = 100

In [ ]:
root_path = '/content/drive/MyDrive/Colab Notebooks/scm-tracking-ml/src/'
root_path

'/content/drive/MyDrive/Colab Notebooks/scm-tracking-ml/src/'

In [ ]:
file_paths = ['data/train/data.xlsx']

li = []
for path in file_paths:
    df = pd.read_excel(root_path + path)
    li.append(df)

data = pd.concat(li, axis=0, ignore_index=True)
data = data.astype(str)
data.tail()

,tracking_number,checkpoint_status,status_description
68624,LY302028146CN,COMPLETED,"KINSTON, NC 28501, Delivered, In/At Mailbox ->..."
68625,LY302028146CN,COMPLETED,"KINSTON, NC 28501, Delivered, In/At Mailbox ->..."
68626,LS364306180CN,COMPLETED,"United States, 【United States】has completed th..."
68627,LS364306180CN,COMPLETED,"United States, 【United States】has completed th..."
68628,LS364306180CN,COMPLETED,"United States, 【United States】has completed th..."


In [ ]:
data.groupby('checkpoint_status').count()

,tracking_number,status_description
checkpoint_status,,
COMPLETED,63115,63115
DELIVERED_GUARANTEE,4,4
IN_US,3147,3147
RETURN_TO_SENDER,1208,1208
TRACKING_AVAILABLE,184,184
TRACKING_ONLINE,971,971


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer

REPLACE_BY_SPACE_RE = re.compile('[/(){}\\[\\]\\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    """
        text: a string
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub(' ', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = re.sub('\\d+', ' ', text) # Remove digit number from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split() ) # Lemmatization

    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

re_tokenizer = RegexpTokenizer(r'\w+')

# Max number of words in each status_description
MAX_SEQUENCE_LENGTH = 250


def tokenizing(df, EMBEDDING_DIM):
    df["tokens"] = df["status_description"].apply(re_tokenizer.tokenize)

    all_words = [word for tokens in df["tokens"] for word in tokens]
    sentence_lengths = [len(tokens) for tokens in clean_data["tokens"]]
    VOCAB = sorted(list(set(all_words)))

    print('Words total: {}'.format(len(all_words)))
    print('Vocabulary size: {}'.format(len(VOCAB)))
    print('Max sentence length is {}'.format(max(sentence_lengths)) )

    # The maximum number of words to be used. (most frequent)
    VOCAB_SIZE = len(VOCAB) + 1
    # OOV = Out of Vocabulary
    oov_tok = '<OOV>' 
    print('VOCAB_SIZE: {}'.format(VOCAB_SIZE))
    print('MAX_SEQUENCE_LENGTH: {}'.format(MAX_SEQUENCE_LENGTH))
    print('EMBEDDING_DIM: {}\n'.format(EMBEDDING_DIM))

    tokenizer = Tokenizer(num_words=VOCAB_SIZE, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~', lower=True)
    tokenizer = Tokenizer(num_words = VOCAB_SIZE, oov_token=oov_tok)
    tokenizer.fit_on_texts(df['status_description'].values)

    return df, tokenizer, VOCAB_SIZE

In [ ]:
def get_embedding_matrix(word_index, VOCAB_SIZE=0, glove_file_path = '/content/drive/MyDrive/Colab Notebooks/Global Vectors for Word Representation (GloVe)/glove.6B.' + str(EMBEDDING_DIM) +'d.txt'):
    f = open(glove_file_path, encoding="utf8")
    embeddings_index = {}

    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))

    embedding_matrix = np.random.random((VOCAB_SIZE, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) !=len(embedding_vector):
                print("could not broadcast input array from shape",str(len(embedding_matrix[i])),
                                 "into shape",str(len(embedding_vector))," Please make sure your"
                                 " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)

            embedding_matrix[i] = embedding_vector

    return embedding_matrix

In [ ]:
# Clean data
data['status_description'] = data['status_description'].apply(clean_text)
clean_data = data
clean_data.tail()

,tracking_number,checkpoint_status,status_description,tokens
68624,LY302028146CN,COMPLETED,kinston nc delivered mailbox item delivered ma...,"[kinston, nc, delivered, mailbox, item, delive..."
68625,LY302028146CN,COMPLETED,kinston nc delivered mailbox item delivered ma...,"[kinston, nc, delivered, mailbox, item, delive..."
68626,LS364306180CN,COMPLETED,united state united state completed investment...,"[united, state, united, state, completed, inve..."
68627,LS364306180CN,COMPLETED,united state united state completed investment...,"[united, state, united, state, completed, inve..."
68628,LS364306180CN,COMPLETED,united state united state completed investment...,"[united, state, united, state, completed, inve..."


In [ ]:
# Tokenizer
clean_data, tokenizer, VOCAB_SIZE = tokenizing(clean_data, EMBEDDING_DIM)
word_index = tokenizer.word_index

Words total: 3750585
Vocabulary size: 7074
Max sentence length is 518
VOCAB_SIZE: 7075
MAX_SEQUENCE_LENGTH: 250
EMBEDDING_DIM: 100



In [ ]:
# Get embedding_matrix
embedding_matrix = get_embedding_matrix(word_index, VOCAB_SIZE)

Total 400000 word vectors.


In [ ]:
# Split data to train set and test set
X = tokenizer.texts_to_sequences(clean_data['status_description'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Y = pd.get_dummies(clean_data['checkpoint_status']).values
print('Shape of label tensor:', Y.shape)

# Split data to train set and test set
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

Shape of data tensor: (68629, 250)
Shape of label tensor: (68629, 6)
(54903, 250) (54903, 6)
(13726, 250) (13726, 6)


In [ ]:
def LSTM():
    from keras.models import Sequential
    from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Conv1D, MaxPooling1D

    model = Sequential()
    model.add(Embedding(input_dim=VOCAB_SIZE,
                        output_dim=EMBEDDING_DIM,
                        weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH,
                        trainable=True))
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(EMBEDDING_DIM, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(6, activation='softmax'))

    return model

In [ ]:
def BILSTM():
    from keras.models import Sequential
    from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Dropout

    model = Sequential()
    model.add(Embedding(input_dim=VOCAB_SIZE,
                        output_dim=EMBEDDING_DIM,
                        weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH,
                        trainable=True))

    model.add(SpatialDropout1D(0.3))
    model.add(Bidirectional(LSTM(EMBEDDING_DIM, dropout=0.3, recurrent_dropout=0.3)))
    model.add(Dense(EMBEDDING_DIM, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(EMBEDDING_DIM, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(6, activation='softmax'))

    return model

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

print('===> kick off training '+model_name+' model')

if model_name == 'lstm':
    model = LSTM()
elif model_name == 'bilstm':
    model = BILSTM()    

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

best_model_file_path = root_path + 'models/'+model_name+'/'+model_name+'_model.h5'
callbacks = [EarlyStopping(monitor='val_loss', verbose=1, patience=2),
             ModelCheckpoint(filepath=best_model_file_path, monitor='val_loss', save_best_only=True)]

history = model.fit(X_train, Y_train, epochs=100, batch_size=256, validation_split=0.1, callbacks=callbacks)

===> kick off training bilstm model
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 100)          707500    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 250, 100)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               160800    
_________________________________________________________________
dense_4 (Dense)              (None, 100)               20100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_3 (Dropout

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Load best model
from keras.models import load_model
best_model = load_model(root_path + 'models/'+model_name+'/'+model_name+'_model.h5')
print('===> Loaded '+model_name+' best model')

In [ ]:
"""
Validate test set
"""
accr = best_model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
"""
Save tokenizer
"""
try:
    tokenizer_file_name = root_path + 'models/'+model_name+'/saved_tokenizers/'+model_name+'_tokenizer_'+ time_now +'.pickel'
    pickle.dump(tokenizer, open(tokenizer_file_name, 'wb'))
    pickle.dump(tokenizer, open(root_path + 'models/'+model_name+'/'+model_name+'_tokenizer.pickel', 'wb'))
    print('===> Saved '+model_name+' tokenizer.')
except Exception as ex:
    print(ex)  

In [ ]:
"""
Save model
"""
try:
    model_file_name = root_path + 'models/'+model_name+'/saved_models/'+model_name+'_model_' + time_now + '.h5'
    best_model.save(model_file_name)
    print('===> Saved '+model_name+' model.')
except Exception as ex:
    print(ex)